In [1]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real
from fffit.fffit.plot import plot_obj_contour

2024-04-16 11:03:16.231232: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 11:03:17.159214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 11:03:17.159357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 11:03:17.261323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-16 11:03:17.491844: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 11:03:17.495696: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
df = pd.DataFrame(columns = ["Molecule", "Property", "Value", "Uncertainty", "Weight"])
for key in list(molec_data_dict.keys()):
    molec_object = molec_data_dict[key]
    for prop_key in list(molec_object.uncertainties.keys()):
        if "vap_density" in prop_key:
            exp_data = molec_object.expt_vap_density
            property_bounds = molec_object.vap_density_bounds
            property_name = "Vapor Density [kg/m^3]"
        elif "liq_density" in prop_key:
            exp_data = molec_object.expt_liq_density
            property_bounds = molec_object.liq_density_bounds
            property_name = "Liquid Density [kg/m^3]"
        elif "Pvap" in prop_key: 
            exp_data = molec_object.expt_Pvap
            property_bounds = molec_object.Pvap_bounds
            property_name = "Vapor pressure [bar]"
        elif "Hvap" in prop_key:
            exp_data = molec_object.expt_Hvap
            property_bounds = molec_object.Hvap_bounds
            property_name = "Enthalpy of Vaporization [kJ/kg]"

        values = np.array(list(exp_data.values()))
        y_unc = molec_object.uncertainties[prop_key]
        weight_unc = 1/(y_unc*values)**2
        weight_2pct = 1/(0.02*values)**2
        weight = np.maximum(weight_unc, weight_2pct)
        df_vals = [key, prop_key, values, y_unc, weight]
        iter_df = pd.DataFrame([df_vals], columns = df.columns)
        iter_df = iter_df.apply(lambda col: col.explode(), axis=0).reset_index(drop=True).copy(deep =True)
        df = pd.concat([df, iter_df], ignore_index=True)
# df.to_csv("molecule_exp_unc_data.csv", index = False, header = True)

In [ ]:
repeats = 1
seed = 1
save_data = False
w_calc = 2
obj_choice = "ExpVal"
at_class = atom_type.AT_Scheme_9()
# molec_data_dict = {"R125":r125_class}
# pm = at_class.get_transformation_matrix(molec_data_dict)
# print(pm)
# theta_paper = np.array([3.599044551, 3.861,	4.00,	1.816694298,	2.957368125,	2.878,	2.997,	
#                         56.52939454,	47.523,	75.000,	5.134160772,	37.41538957,	50.0, 50.0])
# t_paper = theta_paper.reshape(-1,1).T@pm
# print(t_paper)

In [ ]:
# molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
molec_data_dict = {"R125":r125_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
molec_names = list(["R14", "R32", "R50", "R170", "R143a"])
visual.comp_paper_full_ind(molec_names)

In [ ]:
molec_data_dict = {"R32":r32_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, w_calc, obj_choice, save_data)
best_pref = np.array([3.599044551, 3.861,	4.00,	1.816694298,	2.957368125,	2.878,	2.997,	
                        56.52939454,	47.523,	75.000,	5.134160772,	37.41538957,	50.0, 50.0])
best_real= at_optimizer.values_pref_to_real(best_pref)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])
# jac, hess = at_optimizer.approx_jac_hess(best_real)
# print(scipy.linalg.norm(jac, ord = np.inf))

In [ ]:
molec = "R32"
molec_data_dict = {molec:r32_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
# all_gp_dict = {molec : all_gp_dict[molec]}
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, w_calc, obj_choice, save_data)
ls_results = at_optimizer.optimize_ats()
ls_results

In [ ]:
first_param_name = at_optimizer.at_class.at_names[0] + "_min"
last_param_name = at_optimizer.at_class.at_names[-1] + "_min"
molec_best = ls_results.loc[21, first_param_name:last_param_name].values
print(molec_best)

In [ ]:
# Determine the index of the row where column 'B' has the minimum value
min_index = ls_results['Min Obj Cum.'].idxmin()

# Retrieve the row corresponding to the minimum value in column 'B'
min_row = ls_results.iloc[min_index]
print(min_row)

In [ ]:
#Get best values from csvs
def get_best_results(visual, molec_names):
    best_vals = {}
    #Get best_per_run.csv for all molecules
    all_molec_dir = visual.make_results_dir(molec_names)
    all_df = pd.read_csv(all_molec_dir+"/best_per_run.csv", header = 0)
    # print(all_df)
    first_param_name = visual.at_class.at_names[0] + "_min"
    last_param_name = visual.at_class.at_names[-1] + "_min"
    full_opt_best = all_df.loc[0, first_param_name:last_param_name].values
    best_vals["All"] = full_opt_best
    for i in range(len(molec_names)):
        molec_dir = visual.make_results_dir([molec_names[i]])
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        best_vals[molec_names[i]] = molec_best
    return best_vals

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, False, save_data)
best_pref = np.array([3.002,	4.000,	1.500,	3.231,	3.394,	2.927,	38.333,	75.000,	10.000,	50.000,	50.000,	36.598])
# best_pref = ls_results.loc[121, "sigma_C1_min":"epsilon_F_Hx_min"].to_numpy()
# best_real = at_optimizer.values_pref_to_real(best_pref)
best_real= at_optimizer.values_pref_to_real(best_pref)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])
jac, hess = at_optimizer.approx_jac_hess(best_real)
print(scipy.linalg.norm(jac, ord = np.inf))

In [ ]:
molec_data_dict = {"R32":r32_class}#, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))

visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, scl_w, save_data)
best_set = best_pref
visual.plot_obj_hms(best_set)